In [10]:
import torch
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from scipy.spatial import distance

import sys
sys.path.append('/workspaces/dbm25/task_1_2')
from extract_features import extract_features

In [11]:
def top_k_class_prediction(image_path, feature_model, k, measure):
    """
    Predicts the top-k most likely classes for an image using class-wise feature averages.

    Parameters:
    -----------
    image_path : str
        Path to the query image.
    feature_model : str
        The name of the feature layer (e.g., 'layer3').
    k : int
        Number of top predicted classes to return (k <= 2).
    measure : str
        Similarity or distance metric: 'cosine' or 'euclidean'.

    Returns:
    --------
    List of dictionaries:
        Each dict contains:
        - "class": predicted class label
        - "score": similarity or inverse distance score
    """

    assert k<= 3

    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

    # 2. Extract feature for input image
    surrogate_imag_feature= extract_features(image_path,model)
    query_feature = surrogate_imag_feature[feature_model].reshape(1, -1)

    # 3. Load precomputed feature space
    feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")

    # 4. Organize features by class
    class_features = {}
    for item in feature_space:
        class_label = item['class']
        feature_vector = item[feature_model].reshape(1, -1)

        if class_label not in class_features:
            class_features[class_label] = [feature_vector]
        else:
            class_features[class_label].append(feature_vector)

    # 5. Compute mean feature per class
    class_centroids = {}
    for class_label, features in class_features.items():
        stacked = np.vstack(features)
        mean_feature = np.mean(stacked, axis=0).reshape(1, -1)
        class_centroids[class_label] = mean_feature

    # 6. Compute similarity/distance to query
    results = []
    for class_label, centroid in class_centroids.items():
        if measure == "cosine":
            score = cosine_similarity(query_feature, centroid)[0][0]  # Higher = better
        else:  # Euclidean
            score = -np.linalg.norm(query_feature - centroid)  # Lower = better (so negate)

        results.append({
            "class": class_label,
            "score": score
        })

    # 7. Sort and return top-k
    results.sort(key=lambda x: x["score"], reverse=True)
    return results[:k]


In [12]:
results = top_k_class_prediction(
    image_path="/workspaces/dbm25/data/Part2/Part2/brain_tumor/brain_tumor_1007.jpg",
    feature_model="hog",
    k=3,
    measure="cosine"
)

for res in results:
    print(f"Class: {res['class']}, Score: {res['score']:.4f}")


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Processing /workspaces/dbm25/data/Part2/Part2/brain_tumor/brain_tumor_1007.jpg
Class: brain_tumor, Score: 0.8302
Class: brain_menin, Score: 0.7790
Class: brain_glioma, Score: 0.7610
